In [71]:
import os
import random
import pandas as pd
from tkinter import *
from PIL import Image, ImageTk

folder_path = 'D:/desk/JNU/test'  # 图像文件夹路径
num_pairs = 3  # 每轮需要比较的图像对数
num_rounds = 5  # 总共需要比较的轮数
supported_extensions = ['.jpg', '.jpeg', '.png']  # 支持的图像文件扩展名

# 获取所有支持的图片文件列表
images = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f)) and
          os.path.splitext(f)[1].lower() in supported_extensions]

# 创建空字典，用于记录每张图片被选择的次数
selections = {}
for img in images:
    selections[img] = 0

class Application(Frame):
    def __init__(self, master=None):
        try:
            super().__init__(master)
            self.master = master
            self.master.protocol("WM_DELETE_WINDOW", self.on_close)
            self.pack()

            # 初始化变量
            self.round_index = 0
            self.pair_index = 0

            # 生成图片对
            self.generate_pairs()

            # 显示第一轮比较任务
            self.show_images()
        except Exception as e:
            print('An error occurred:', e)
            self.save_scores()
            self.master.destroy()

    def show_images(self):
        global pairs
        if self.round_index >= num_rounds:
            # 所有比较完成，保存结果并退出
            self.save_scores()
            self.master.destroy()
            return

        if self.pair_index >= num_pairs:
            # 一轮比较完成，保存结果并重新生成新的任务对
            self.save_scores()
            self.generate_pairs()
            self.pair_index = 0
            self.round_index += 1

        # 获取当前对比的两张图像
        pair = pairs[self.pair_index]
        if len(pair) != 2:
            # 图像对不完整，退出程序
            self.save_scores()
            self.master.destroy()
            return

        img1 = Image.open(os.path.join(folder_path, pair[0]))
        img2 = Image.open(os.path.join(folder_path, pair[1]))
        img1 = img1.resize((img1.width // 4, img1.height // 4))
        img2 = img2.resize((img2.width // 4, img2.height // 4))

        # 添加文本
        text_label = Label(self, text="Q1：哪张图更让你感觉到安全？",font=("Microsoft YaHei UI", 12))
        text_label.grid(row=5, column=0, columnspan=5)

        # 显示图像
        tkimg1 = ImageTk.PhotoImage(img1)
        tkimg2 = ImageTk.PhotoImage(img2)
        self.label1 = Label(self, image=tkimg1)
        self.label1.image = tkimg1
        self.label1.grid(row=0, column=0)
        self.label2 = Label(self, image=tkimg2)
        self.label2.image = tkimg2
        self.label2.grid(row=0, column=1)

        # 添加选择按钮
        self.button1 = Button(self, text='选择左边的图片', command=lambda: self.choose_image(pair[0]))
        self.button1.grid(row=1, column=0)
        self.button2 = Button(self, text='选择右边的图片', command=lambda: self.choose_image(pair[1]))
        self.button2.grid(row=1, column=1)

    def generate_pairs(self):
        global pairs
        pairs = []
        while len(pairs) < num_pairs:
            pair = random.sample(images, 2)
            if pair not in pairs:
                pairs.append(pair)

    def choose_image(self, img):
        # 增加当前图片的被选择次数
        selections[img] += 1

        # 显示下一对图像
        self.pair_index += 1
        self.label1.destroy()
        self.label2.destroy()
        self.button1.destroy()
        self.button2.destroy()
        self.show_images()

    def save_scores(self):
        scores = []
        for img in images:
            scores.append(selections[img])

        df = pd.DataFrame({'image': images, 'score': scores})
        #df.to_csv('D:/desk/JNU/scores.csv', index=False)
        return df

    def on_close(self):
        try:
            self.save_scores()
            self.master.destroy()
        except Exception as e:
            print('An error occurred:', e)
            self.master.destroy()

# 创建 GUI 程序并运行
root = Tk()
app = Application(master=root)
app.mainloop()

In [37]:
app.save_scores()

,image,score
0,1_out.jpg,2
1,2_out.jpg,2
2,3_out.jpg,3


In [43]:
import pandas as pd
from trueskill import Rating, rate_1vs1

# 读取图片得分数据
df = app.save_scores()

# 初始化等级分
ratings = {}
for img in df['image']:
    ratings[img] = Rating()

# 计算每张图片的等级分
for i, row in df.iterrows():
    img1 = row['image']
    score1 = row['score']
    r1 = ratings[img1]
    for img2 in ratings.keys():
        if img2 == img1:
            continue
        r2 = ratings[img2]
        score2 = df.loc[df['image'] == img2]['score'].item()
        if score1 > score2:
            r1, r2 = rate_1vs1(r1, r2)
        elif score1 < score2:
            r2, r1 = rate_1vs1(r2, r1)
        ratings[img1], ratings[img2] = r1, r2

# 输出每张图片的等级分
for img, r in ratings.items():
    print('{}: {:.2f}'.format(img, r.mu))        
        
# 映射等级分到 0~10 的浮点值
min_val = min([r.mu - 3 * r.sigma for r in ratings.values()])
max_val = max([r.mu + 3 * r.sigma for r in ratings.values()])
for img, r in ratings.items():
    val = (r.mu - min_val) / (max_val - min_val) * 5   #调整输出值的区间0-10
    df.loc[df['image'] == img, 'score'] = val

# 打印每张图片的归一化分数值
print(df)
    
# 保存到 CSV 文件中
df.to_csv('D:/desk/JNU/test/scores_normalized.csv', index=False)

1_out.jpg: 19.19
2_out.jpg: 20.15
3_out.jpg: 34.08
       image     score
0  1_out.jpg  1.910131
1  2_out.jpg  2.003402
2  3_out.jpg  3.349441


In [44]:
df.head()

,image,score
0,1_out.jpg,1.910131
1,2_out.jpg,2.003402
2,3_out.jpg,3.349441
